In [ ]:
import requests
import datetime
import math

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [ ]:
# Clean values inside the td tag
def clean(v):
    return v.replace('\t', '').replace('\r', '').replace('\n', ' ').replace(u'\xa0', '').replace('--', '')

In [ ]:
# Check if value is nan or empty string
def is_empty(v):
    return (type(v) == float and math.isnan(v)) or (type(v) == str and len(v.strip()) == 0)

In [ ]:
# Request the html page and webscrap for data
# Returns a data frame
def get_data(date, station):
    
    station_name = station.split('/')[1]
    
    date_str = date.strftime('%d/%m/%Y')

    form_input = {'texData': f'{date_str}', 'selEst': f'{station}'}

    page = requests.post(URL, data = form_input)

    soup = BeautifulSoup(page.content, 'html.parser')
    
    parsed_data = {'Date': date_str, 'Station': station_name}

    for table in soup.find_all('table', class_='font01')[1:]:
        data = []
        for tr in table.find_all('tr'):
            tds = tr.find_all('td')
            data.append(clean(tds[len(tds) - 1].text))

        parsed_data[(data[0] + ' ' + data[1]).strip()] = data[2:]
    
    df = pd.DataFrame(parsed_data)
    
    # Filter empty rows
    df = df[df.applymap(is_empty).sum(axis=1) < (df.shape[1] - 3)]
    
    return df

In [ ]:
# Air quality url
URL = 'https://sistemasinter.cetesb.sp.gov.br/Ar/php/ar_dados_horarios_resultado.php'

In [ ]:
STATIONS = [
    '65/Americana',
    '72/Araçatuba',
    '71/Araraquara',
    '73/Bauru',
    '42/Campinas-Centro',
    '41/Campinas-Taquaral',
    '43/Campinas-V.União',
    '37/Capão Redondo',
    '28/Carapicuíba',
    '81/Catanduva',
    '10/Cerqueira César',
    '31/Cid.Universitária-USP-Ipen',
    '8/Congonhas',
    '24/Cubatão-Centro',
    '25/Cubatão-V.Parisi',
    '30/Cubatão-Vale do Mogi',
    '15/Diadema',
    '29/Grajaú-Parelheiros',
    '62/Guaratinguetá',
    '35/Guarulhos-Paço Municipal',
    '40/Guarulhos-Pimentas',
    '5/Ibirapuera',
    '34/Interlagos',
    '33/Itaim Paulista',
    '50/Itaquera',
    '54/Jacareí',
    '75/Jaú',
    '74/Jundiaí',
    '64/Limeira',
    '36/Marg.Tietê-Pte Remédios',
    '76/Marília',
    '22/Mauá',
    '3/Mooca',
    '6/N.Senhora do Ó',
    '17/Osasco',
    '1/Parque D.Pedro II',
    '44/Paulínia',
    '48/Paulínia-Sta Terezinha',
    '66/Perus',
    '39/Pico do Jaraguá',
    '27/Pinheiros',
    '77/Piracicaba',
    '78/Presidente Prudente',
    '84/Ribeirão Preto',
    '59/Rio Claro-Jd.Guanabara',
    '18/S.André-Capuava',
    '38/S.Bernardo-Centro',
    '19/S.Bernardo-Paulicéia',
    '55/S.José Campos',
    '56/S.José Campos-Jd.Satelite',
    '57/S.José Campos-Vista Verde',
    '53/Santa Gertrudes',
    '2/Santana',
    '16/Santo Amaro',
    '82/Santos',
    '83/Santos-Ponta da Praia',
    '7/São Caetano do Sul',
    '80/São José do Rio Preto',
    '51/Sorocaba',
    '20/Taboão da Serra',
    '94/Tatuí',
    '58/Taubaté'
]

In [ ]:
begin_date_str = '2013-05-08'
end_date_str = '2020-04-30'

In [ ]:
begin_date = datetime.datetime.strptime(begin_date_str, '%Y-%m-%d').date()
end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d').date()

dates = pd.date_range(begin_date, end_date).tolist()

In [ ]:
air_quality = pd.DataFrame(columns=[
        'Date', 
        'Hora', 
        'Station', 
        'BENZENO ug/m3 Média Horária', 
        'CO ppm Índice / Qualidade',
        'MP10 µg/m³ Índice / Qualidade',
        'MP2.5 µg/m³ Índice / Qualidade', 
        'NO2 µg/m³ Índice / Qualidade',
        'O3 µg/m³ Índice / Qualidade', 
        'SO2 µg/m³ Índice / Qualidade',
        'TOLUENO ug/m3 Média Horária', 
        'TRS ppb Média Horária'])

# Get data for all dates and stations
for date in dates:
    for station in STATIONS:
        data = get_data(date.date(), station)
        air_quality = air_quality.append(data, ignore_index=True)

In [ ]:
# Rename columns and save
air_quality.columns = [
        'Date', 
        'Hour', 
        'Station', 
        'Benzene', 
        'CO',
        'PM10',
        'PM2.5', 
        'NO2',
        'O3', 
        'SO2',
        'Toluene', 
        'TRS']

In [ ]:
def convert_float(v):
    if type(v) == str and ',' in v:
        return float(v.replace(',', '.'))
    return v

In [ ]:
# Convert date and time to a single field

air_quality.loc[air_quality['Hour'] == '24:00', 'Hour'] = '00:00'

air_quality['DatetimeStr'] = air_quality['Date'] + ' ' + air_quality['Hour']

air_quality['Datetime'] = pd.to_datetime(air_quality['DatetimeStr'], format='%d/%m/%Y %H:%M')

In [ ]:
final = air_quality[['Datetime', 'Station']]
final[['Benzene', 'CO', 'PM10', 'PM2.5', 'NO2', 'O3', 'SO2', 'Toluene', 'TRS']] = air_quality[['Benzene', 'CO', 'PM10', 'PM2.5', 'NO2', 'O3', 'SO2', 'Toluene', 'TRS']].applymap(convert_float)

In [ ]:
final.to_csv('data/sp_air_quality.csv', index=False)